In [1]:
# -*- coding: UTF-8 -*-
import os
import re
import csv
import sys
import json
import datetime
import requests
import difflib
RAW_DIR = "../raw/"
LIST_RAW_CSV = "../list_raw/all.csv"
LIST_RAW_JSON = "../transformed/all_json.json"
TRANSFORMED_DIR = "../transformed/"
OUTPUT = "output.txt"
GBTOUTPUT = "gbt.txt"
TITLEOUTPUT = "title.txt"

In [2]:
#
full_keys = ["author","title","journal","year","DOI","month","citations(google scholar)","abstract","keywords","reference_count","ccfClass","important","references"]
full_keys_default = {"author":[],"title":"","journal":"","year":0,"DOI":"","month":0,"citations(google scholar)":-1,"abstract":"","keywords":[],"reference_count":0,"ccfClass":"","important":None,"references":[]}
# 常用函数定义
# 调用crossref接口
def string_similar(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).ratio()
def get_crossref_info(ref):
    url = "https://doi.crossref.org/servlet/query"

    querystring = {"usr":"halloweenwx@163.com","pwd":"halloween","format":"json","qdata":"""<?xml version = "1.0" encoding="UTF-8"?>
    <query_batch version="2.0" xmlns = "http://www.crossref.org/qschema/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
      <head>
          <email_address>hisham@atypon.com</email_address>
          <doi_batch_id>Sample multi resolve</doi_batch_id>
      </head>
      <body>
           <query key="mykey" enable-multiple-hits="true">
      <unstructured_citation>"""+ref+"""</unstructured_citation>
    </query>
         </body>
    </query_batch>"""}

    headers = {
        'User-Agent': "PostmanRuntime/7.20.1",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "123bedd6-8fb9-43e5-980a-54336e7aa684,6312f073-8e24-4222-abb2-789899bbc01f",
        'Host': "doi.crossref.org",
        'Accept-Encoding': "gzip, deflate",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
#     print(response.text)
    return response.text

# 保存至transform
def save_tran_json(addr,jsonfile):
    with open(TRANSFORMED_DIR + addr, 'w') as o:
        o.write(json.dumps(jsonfile,ensure_ascii=False))
        
def save_tran_plain(addr,txtfile):
    with open(TRANSFORMED_DIR + addr, 'w') as o:
        o.write(txtfile)
        
def save_tran_lines(addr,txtfile):
    with open(TRANSFORMED_DIR + addr, 'a+') as o:
        for line in txtfile:
            o.write(line+'\n')    
# 从transformd中读取
def read_tran_json(addr):
    with open(TRANSFORMED_DIR + addr,encoding = 'utf-8') as f:
        data = json.load(f)
    return data
    
#  查询ccf等级
def search_ccf(s):
    ccf_all_addr = "/Users/Halloween/Desktop/Study/复杂网络/lrcns/raw-data/ccf/ccf_all.csv"

    ccf_all = []
    ccf_search_res = {}
    with open(ccf_all_addr,encoding = 'utf-8') as f:
        reader = csv.reader(f)
        for line in reader:
            ccf_all.append(line)
    level = ""
    field = ""
    cORj = ""
    col = -1
#     print(ccf_all)
    for line in reversed(ccf_all):
        if col != 1 and col != 2 and col != -1:
            col = -1
            continue
        if col == -1:
            for word in line[1:3]:
                if word == "":
                    continue
                if(string_similar(word,s)>0.9):
                    col = line.index(word)
                    ccf_search_res['ccf_search_res'] = word
#                     print('"'+word.strip('"')+'"'+'"'+s.strip('"')+'"')
#                     print(col)
#                     print(line)
                    break
        else:
            if len(line[0])>1 and line[0][1]=='、' and level == "":
                level = line[0][2]
#                 print(level)
            if len(line[0])>10 and line[0][:2]=="中国" and field == "":
                field = line[0][line[0].find("（")+1:line[0].find("）")]
                cORj = line[0][line[0].find("术")+1:line[0].find("（")]
#                 print(field)
    ccf_search_res.update({"level":level,"field":field,"type":cORj})
    return {"level":level,"field":field,"type":cORj}
# 数据审查函数
def inspect(to_inspect):
    papers_arr = to_inspect
    all_error = 0
    for paper in papers_arr:
        inspect = []
        for key in full_keys:
            if(key not in paper.keys() or type(paper[key]) != type(full_keys_default[key]) or paper[key] == full_keys_default[key]):
                inspect.append(key + " lost")
                continue
            if(key == "author" or key == "keywords" or key == "references"):
                if(type(paper[key]) != list or len(paper[key])==0):
                    inspect.append(key + " lost")
                    continue
            if(key == "year"):
                if(paper[key]<1900):
                    inspect.append(key + " error")
            if(key == "month"):
                if(paper[key]<0 or paper[key] >12):
                    inspect.append(key + " error")
            if(key == "citations(google scholar)" or key == "reference_count"):
                if(paper[key]<0):
                    inspect.append(key + " error")
            if(key == 'ccfClass'):
                if(paper[key] not in ['A','B','C']):
                    inspect.append(key + " error")
            if(type(paper[key])==int):
                if(paper[key]<0):
                    inspect.append(key + " less than 0")
                continue
            if(type(paper[key])==float):
                inspect.append(key + " not int")
                continue
        paper['inspect'] = list(inspect)
        print(paper['title']+str(paper['inspect']))
        all_error += len(paper['inspect'])
    print(">>>>SUMMARY ERRORs:"+str(all_error)+"<<<<<")
    return papers_arr

In [9]:
# 路径解析
def init_from_raw():
    addr_list = os.listdir(RAW_DIR)
    for fname in list(addr_list):
        addr = RAW_DIR + fname
        if(fname[0]=='.' or addr == OUTPUT or os.path.isdir(addr)):
            addr_list.remove(fname)
            print("skip " + fname )
            continue
    print(len(addr_list))

    # 数据读入内存 all_arr
    all_arr = []
    for fname in addr_list:
        addr = RAW_DIR + fname
        with open(addr,'r', encoding='UTF-8-sig') as f:
            farr = json.load(f)
            print(fname)
            en_topic,cn_topic = fname.replace(".txt","").split("_")
            for paper in farr:
                paper['author'] = [y.strip('\ufeff') for y in paper['author']]
                if(type(paper['DOI'])==str):
                    paper['DOI'] = paper['DOI'].replace("https://doi.org/","")
                    if(paper['DOI'][:2]!="10"):
                        paper['DOI'] = full_keys_default["DOI"]
                else:
                    paper['DOI'] = full_keys_default["DOI"]
                paper["topic"] = {"en":en_topic,"cn":cn_topic}
    #         max_title = 0
    #         max_title_self = ""
    #         for article in farr:
    #             if max_title < len(article['title']):
    #                 max_title = len(article['title'])
    #                 max_title_self = article['title']
    #         print("max_title: " + str(max_title))
    #         print(max_title_self)
            all_arr.extend(list(farr))
    print(len(all_arr))
    return all_arr

In [195]:
def read_list_raw():
    res = []
    with open(LIST_RAW_CSV,'r') as f: 
        res = [x.strip('\n') for x in f.readlines()]
    return res

In [196]:
def read_list_raw_json():
    with open(LIST_RAW_JSON,'r') as f: 
        raw_json = json.load(f)
        return raw_json

In [10]:
all_arr = init_from_raw()
list_arr = read_list_raw()
raw_json = read_list_raw_json()

skip .DS_Store
skip 动态嵌入（已合并至上层）
10
Multimedia Social Relationship Network_多媒体社交关系网络研究.txt
Social Influence Prediction_社会影响预测.txt
Information Diffusion Detection_信息扩散预测.txt
Dynamic Embedding (Temporal)_动态嵌入.txt
Knowledge Graph Embedding_知识图谱.txt
network embedding_网络嵌入.txt
Network  Fusion_网络融合.txt
Community Detection_社团检测.txt
Graph based Recommendation_基图推荐.txt
GraphVite_GV系统介绍.txt
402


NameError: name 'read_list_raw' is not defined

In [219]:
print(len(all_arr))

402


In [225]:
print(json.dumps(cleaned_arr[0]))

{"author": ["Scott A.Golder"], "title": "Measuring Social Networks with Digital Photograph Collections", "journal": "ACM Conference on Hypertext and Hypermedia", "year": 2008, "DOI": "10.1145/1379092.1379104", "month": 6, "citations(google scholar)": 37, "abstract": "The ease and lack of cost associated with taking digital photographs have allowed people to amass large personal photograph collections. These collections contain valuable information about their owners' social relationships. This paper is a preliminary investigation into how digital photo collections can provide useful data for the study of social networks. Results from an analysis of 23 subjects\u2019 photo collections demonstrate the feasibility of this approach. The relationship between perceived closeness and network position, as well as future questions, are also discussed.", "keywords": ["Social Networks", "Photographs."], "reference_count": 16, "ccfClass": "", "important": true, "references": [{"ref": "[1] Adamic, 

In [203]:
# 整理清单论文
list_tran_arr = []
obj = {}
print (list_arr[0])
print (list_arr[0].split(','))
for record in list_arr:
    author_start = 1
    record_list = record.split(',')

    obj['title'] = record_list[0].strip('"')
    if(record_list[0][0]=='"'):
        for part in record_list[1:]:
            obj['title'] += part.strip('"')
            author_start += 1
            if(part[-1]!='"'):
                break
    obj['author'] = [x.strip('"') for x in record_list[author_start:-3]]
    obj['journal'] = record_list[-3]
    obj['year'] = int(record_list[-2])
    obj['important'] = bool(record_list[-1])
    list_tran_arr.append(dict(obj))
save_tran_json("list_tran_arr.txt",list_tran_arr)

A Comparative Study between "Prediction-response" and "Scenario-response" in Unconventional Emergency Decision-making Management,"Pang Jiaju,Li Shiming,Liu Liang",IEEE,2011,TRUE
['A Comparative Study between "Prediction-response" and "Scenario-response" in Unconventional Emergency Decision-making Management', '"Pang Jiaju', 'Li Shiming', 'Liu Liang"', 'IEEE', '2011', 'TRUE']


In [202]:
all_title = []
for x in all_arr:
    all_title.append(x['title'])
save_tran_lines('all_title.txt',all_title)

In [204]:
wos_arr = read_tran_json('wos.txt')

In [212]:
for paper in wos_arr:
    try:
        print(paper['DI'])
        if(paper['DI'][0] in ref_doi_set):
            print(">>>>>>>")
    except:
        print("")

['10.1109/GreenCom.2012.72']
['10.3233/IDA-2011-0486']
>>>>>>>
['10.1109/TMM.2012.2187436']
>>>>>>>
['10.1109/CVPR.2013.320']
>>>>>>>
['10.1109/ICCV.2017.269']
>>>>>>>
['10.1007/s11263-017-1055-1']
>>>>>>>
['10.3233/JIFS-169199']
['10.1016/j.dss.2018.09.006']
['10.7305/automatika.2016.01.880']
['10.1109/WACV.2015.136']
>>>>>>>
['10.3233/SW-140134']

['10.1016/j.neucom.2016.12.024']




['10.1109/TMM.2015.2461462']
>>>>>>>
['10.1109/TKDE.2017.2672749']
['10.1109/JCN.2015.000026']
['10.1109/ICCV.2017.289']
>>>>>>>
['10.1016/j.patcog.2017.03.001']
>>>>>>>
['10.1109/TKDE.2018.2822283']
>>>>>>>
['10.1007/978-3-319-73603-7_29']
>>>>>>>
['10.1109/TKDE.2018.2872602']
['10.1145/2911451.2914713']
>>>>>>>



['10.5220/0006749206650674']
['10.1007/s11263-017-1055-1']
>>>>>>>
['10.1016/j.patcog.2017.03.001']
>>>>>>>
['10.1145/3292500.3330989']
['10.1016/j.ssresearch.2018.04.004']
['10.1002/bdm.625']
['10.1109/ACCESS.2018.2839770']
>>>>>>>
['10.1109/ICRTCCM.2017.82']
['10.1109/ICSC.2019.00042']

['1

In [218]:
union_arr = []
def do_union():
    union_arr.extend(all_arr)
    for list_record in list_tran_arr:
#         print("<<<"+temp_record['title'])
        sim = 0;
        for union_record in list(union_arr):
#             print(string_similar(temp_record['title'],union_record['title']))            
            sim = max(sim,string_similar(list_record['title'],union_record['title']))
        if(sim<0.92):
            union_arr.append(dict(list_record))
    for record in union_arr:
        for key in full_keys:
            if(key not in record.keys()):
                record[key] = full_keys_default[key]
                
    return union_arr

union_arr = do_union()
# print(len(union_arr))

save_tran_json('union.txt',union_arr)
union_arr = read_tran_json('union.txt')
print(len(union_arr))

498


In [115]:
for x in union_arr:
    print(x.keys())

dict_keys(['author', 'title', 'journal', 'year', 'DOI', 'month', 'citations(google scholar)', 'abstract', 'keywords', 'reference_count', 'ccfClass', 'important', 'references', 'topic', 'is_important', 'inspect', 'search_ccf'])
dict_keys(['author', 'title', 'journal', 'year', 'DOI', 'month', 'citations(google scholar)', 'abstract', 'keywords', 'reference_count', 'ccfClass', 'important', 'references', 'topic', 'is_important', 'inspect', 'search_ccf'])
dict_keys(['author', 'title', 'journal', 'year', 'DOI', 'month', 'citations(google scholar)', 'abstract', 'keywords', 'reference_count', 'ccfClass', 'important', 'references', 'topic', 'is_important', 'inspect', 'search_ccf'])
dict_keys(['author', 'title', 'journal', 'year', 'DOI', 'month', 'citations(google scholar)', 'abstract', 'keywords', 'reference_count', 'ccfClass', 'important', 'references', 'topic', 'is_important', 'inspect', 'search_ccf'])
dict_keys(['author', 'title', 'journal', 'year', 'DOI', 'month', 'citations(google scholar)'

In [99]:
union_title_set = set()
for x in union_arr:
    try:
        if(x['DOI'][:2]=="10"):
            union_title_set.add(x['DOI'])
    except:
        print(x['DOI'])
    
print(len(union_title_set))
print(union_title_set)

1603.07704
261
{'10.1145/1401890.1402008·Source:DBLP', '10.1145/2806416.2806502', '10.1145/3041021.3053380', '10.11896/jsjkx.180901759', '10.1109/ICDE.2019.00061', '10.3233/JIFS-169199', '10.1007/s11263-017-1055-1', '10.1109/ICCV.2017.289', '10.1109/ICCV.2017.269', '10.1145/3132847.3132925', '10.1109/TKDE.2017.2783933', '10.1007/978-3-642-37401-2_32', '10.1145/1376616.1376746', '10.1145/3219819.3219988', '10.1016/j.patcog.2019.107000', '10.1109/access.2019.2942221', '10.1002/bdm.625', '10.1016/j.ins.2019.09.022', '10.1109/TKDE.2018.2822283', '10.1016/j.neucom.2016.11.054', '10.3233/SW-140134', '10.3115/v1/D14-1067', '10.1145/3184558.3191526', '10.1109/TKDE.2019.2941938', '10.18653/v1/D16-1260', '10.1145/3350546.3360738', '10.1016/j.procs.2017.05.045', '10.1145/3209978.3209981', '10.5220/0006749206650674', '10.1109/CVPR.2012.6248095', '10.1109/ACCESS.2018.2832087', '10.1145/3308560.3316585', '10.24963/ijcai.2018/288', '10.1016/j.jocs.2017.11.015', '10.1109/icde.2017.35', '10.1145/321981

In [10]:
inspect_union_arr = inspect(union_arr)

Measuring Social Networks with Digital Photograph Collections['ccfClass lost', 'is_important lost']
CLOSE & CLOSER: DISCOVER SOCIAL RELATIONSHIP FROM PHOTO COLLECTIONS['is_important lost']
Close & Closer: Social Cluster and Closeness from Photo Collections['is_important lost']
Understanding Kin Relationships in a Photo['is_important lost']
Discovering Informative Social Subgraphs and Predicting Pairwise Relationships from Group Photos['is_important lost']
Who Do I Look Like? Determining Parent-Offspring Resemblance via Gated Autoencoders['is_important lost']
Graph-based Kinship Recognition['is_important lost']
Tri-Subject Kinship Verification: Understanding the Core of A Family['is_important lost']
Family Member Identification from Photo Collections['ccfClass lost', 'is_important lost']
Visual Kinship Recognition of Families in the Wild['is_important lost']
Learning deep compact similarity metric for kinship verification from face images['ccfClass lost', 'is_important lost']
Learning p

In [11]:
for paper in inspect_union_arr:
    for ins in paper['inspect']:
        if("ccfClass" in ins):
            print(paper['title'] + " " + str(paper['inspect']))

Measuring Social Networks with Digital Photograph Collections ['ccfClass lost', 'is_important lost']
Family Member Identification from Photo Collections ['ccfClass lost', 'is_important lost']
Learning deep compact similarity metric for kinship verification from face images ['ccfClass lost', 'is_important lost']
Automatic Social Network Construction from Movies Using Film-Editing Cues ['ccfClass lost', 'is_important lost']
Character Relationship Analysis in Movies Using Face Tracks ['DOI lost', 'ccfClass lost', 'is_important lost']
Active Clustering with Ensembles for Social Structure Extraction ['ccfClass lost', 'is_important lost']
Social Network Analysis of TV Drama Characters via Deep Concept Hierarchies ['ccfClass lost', 'is_important lost']
Social Network Construction of the Role Relation in Unstructured Data Based on Multi-view ['ccfClass lost', 'is_important lost']
StoryRoleNet: Social Network Construction of Role Relationship in Video ['ccfClass lost', 'is_important lost']
SRE-

In [12]:
def add_data():
    temp_arr = inspect_union_arr
    for paper in temp_arr:
        paper['search_ccf'] = search_ccf(paper['journal'])
        for ins in paper['inspect']:
            if('ccfClass' in ins):
                paper['ccfClass'] = paper['search_ccf']['level']                
    return temp_arr

In [13]:
added_inspect_union_arr = add_data()
for x in added_inspect_union_arr:
#     if(x['ccfClass']!=x['search_ccf']['level']):
    print(x['journal']+"-->"+x['ccfClass']+str(x['search_ccf']))

ACM Conference on Hypertext and Hypermedia-->{'level': '', 'field': '', 'type': ''}
ICME-->B{'level': 'B', 'field': '计算机图形学与多媒体', 'type': '会议'}
MM-->A{'level': '', 'field': '', 'type': ''}
IEEE MM-->B{'level': '', 'field': '', 'type': ''}
MM-->A{'level': '', 'field': '', 'type': ''}
CVPR-->A{'level': 'A', 'field': '人工智能', 'type': '会议'}
ICPR-->C{'level': 'C', 'field': '人工智能', 'type': '会议'}
IEEE MM-->B{'level': '', 'field': '', 'type': ''}
WACV-->{'level': '', 'field': '', 'type': ''}
IEEE TRANSACTIONS ON PATTERN ANALYSIS AND MACHINE INTELLIGENCE-->A{'level': '', 'field': '', 'type': ''}
Information Fusion-->{'level': '', 'field': '', 'type': ''}
Pattern Recognition Letters-->C{'level': 'C', 'field': '人工智能', 'type': '期刊'}
ICCV-->A{'level': 'A', 'field': '人工智能', 'type': '会议'}
CVPR-->A{'level': 'A', 'field': '人工智能', 'type': '会议'}
ICCV-->A{'level': 'A', 'field': '人工智能', 'type': '会议'}
IJCV-->A{'level': 'A', 'field': '人工智能', 'type': '期刊'}
IJCAI-->A{'level': 'A', 'field': '人工智能', 'type': '会议'}

In [14]:
with open(TRANSFORMED_DIR + "c.txt",'a+') as o:
    o.write("[")

In [16]:
# doii = 0
# cnt = 0
# fail = 0
# with open(TRANSFORMED_DIR + "c.txt",'a+') as o:
#     for x in added_inspect_union_arr:
#         print(".")
#         for p in x['references']:
#             if p['ref'].lower().find('doi') != -1:
#                 doii += 1
#             else:
#                 try:
#                     p['cross'] = get_crossref_info(p['ref'])
#                     cnt += 1
#                     o.write(json.dumps(x,ensure_ascii=False))                    
#                     o.write(',')
#                     print(cnt)
#                 except:
#                     print("ERROR:"+str(fail))
#                     fail += 1
#     print(doii)
# #         print(get_crossref_info(p['ref']))
# #         if p['ref'].lower().find('doi'):
# #             print(p['ref']) 

In [26]:
ctxt = ""
with open(TRANSFORMED_DIR + "c.txt",'r') as f:
    ctxt = f.read()
#     yarr = []
    print((arrt[949631:])[0])    
    print((arrt[949631:])[-1])
#     arr = json.loads(arrt[949631:])
#     print(len(arr))
#     xt = "Measuring Social Networks with Digital Photograph Collections"
#     xf = {}
#     for x in arr:
#         print(x['title']+str(len(str(x['references']))))
#         if(xt != x['title']):
#             xt = x['title']
#             yarr.append(dict(xf))
#             xf = x
#         else:
#             xf = x
#     yarr.append(dict(xf))
#     for y in yarr:
#         print(y['title']+str(len(str(y['references']))))
#     save_tran_json('y.txt',y)

[
,


In [33]:
c_clean = ctxt[949631:][:-1]+']'
print(c_clean[0])
print(c_clean[-1])
print(ctxt[-1])
print(len(ctxt))

print((ctxt[949631:])[0])
print((ctxt[949631:])[-1])

[
]
,
374025383
[
,


In [34]:
c_json = json.loads(c_clean)

In [38]:
xt = "Measuring Social Networks with Digital Photograph Collections"
xf = {}
yarr = []
print(len(c_json))
for x in c_json:
#         print(x['title']+str(len(str(x['references']))))
    if(xt != x['title']):
        xt = x['title']
        yarr.append(dict(xf))
        xf = x
    else:
        xf = x
# 加入最后一个
yarr.append(dict(xf))
for y in yarr:
    print(y['title']+str(len(str(y['references']))))
print(len(yarr))
save_tran_json('y_clean.txt',yarr)

# print(x['title']+str(len(str(x['references']))))

10253
Measuring Social Networks with Digital Photograph Collections15276
CLOSE & CLOSER: DISCOVER SOCIAL RELATIONSHIP FROM PHOTO COLLECTIONS5275
Close & Closer: Social Cluster and Closeness from Photo Collections5042
Understanding Kin Relationships in a Photo58688
Discovering Informative Social Subgraphs and Predicting Pairwise Relationships from Group Photos22018
Who Do I Look Like? Determining Parent-Offspring Resemblance via Gated Autoencoders27542
Graph-based Kinship Recognition19726
Tri-Subject Kinship Verification: Understanding the Core of A Family59374
Family Member Identification from Photo Collections26938
Visual Kinship Recognition of Families in the Wild95965
Learning deep compact similarity metric for kinship verification from face images30945
Learning part-aware attention networks for kinship verification8462
Learning Social Relation Traits from Face Images50533
A Domain Based Approach to Social Relation Recognition63254
Dual-Glance Model for Deciphering Social Relationsh

In [85]:
x_title = set()
for paper in yarr:
    print(paper['DOI'])
    x_title.add(paper['title'])
print(len(x_title))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [86]:
cleaned_arr = read_tran_json('y_clean.txt')

In [101]:
cleaned_title_set = set()
for paper in cleaned_arr:
    cleaned_title_set.add(paper['title'])
print(len(cleaned_title_set))

286


In [214]:
print(cleaned_doi_set)

{'Dynamic Network Embedding via Incremental Skip-gram with Negative Sampling', '10.11896/jsjkx.180901759', '10.1109/ICDE.2019.00061', '10.1007/s11263-017-1055-1', '10.1109/TKDE.2017.2783933', 'On Exploring Semantic Meanings of Links for Embedding Social Networks', '10.1007/978-3-642-37401-2_32', 'ARROW: Approximating Reachability Using Random Walks Over Web-Scale Graphs', 'Easing Embedding Learning by Comprehensive Transcription of Heterogeneous Information Networks', '10.1016/j.ins.2019.09.022', '10.3115/v1/D14-1067', 'BL-MNE: Emerging heterogeneous social network embedding through broad learning with aligned autoencoder', 'Tri-Subject Kinship Verification: Understanding the Core of A Family', 'DepthLGP: Learning Embeddings of Out-of-Sample Nodes in Dynamic Networks', 'A three-stage algorithm on community detection in social networks', '10.1145/3350546.3360738', 'Continuous-Time Dynamic Network Embeddings', '10.5220/0006749206650674', 'A divide and agglomerate algorithm for community 

In [102]:
combine_union_cleaned_arr = []
combine_union_cleaned_arr.extend(cleaned_arr)
for paper in union_arr:
    if(paper['title'] not in cleaned_title_set):
        combine_union_cleaned_arr.append(paper)
print(len(combine_union_cleaned_arr))

484


In [105]:
for paper in combine_union_cleaned_arr:
    print(paper['author'])

['Scott A.Golder']
['Peng Wu', 'Weimin Ding', 'Zhidong Mao', 'Dan Tretter']
['Peng Wu', 'Dan Tretter']
['Siyu Xia', 'Ming Shao', 'Jiebo Luo', 'Yun Fu']
['Yan-Ying Chen', 'Winston H. Hsu', 'Hong-Yuan Mark Liao']
['Afshin Dehghan', 'Enrique G. Ortiz', 'Ruben Villegas', 'Mubarak Shah']
['Yuanhao Guo， Hamdi Dibeklio˘glu， Laurens van der Maaten']
['Xiaoqian Qin', 'Xiaoyang Tan', 'and Songcan Chen']
['Qieyun Dai', 'Peter Carr', 'Leonid Sigal', 'Derek Hoiem']
['Joseph P. Robinson', 'Ming Shao', 'Yue Wu', 'Hongfu Liu', 'Timothy Gillis', 'Yun Fu']
['Xiuzhuang Zhou', 'Kai Jin', 'Min Xu', 'Guodong Guo']
['Haibin Yan', 'Shiwei Wang']
['Zhanpeng Zhang', 'Ping Luo', 'Chen Change Loy', 'Xiaoou Tang']
['Qianru Sun', 'Bernt Schiele', 'Mario Fritz']
['Junnan Li', 'Yongkang Wong', 'Qi Zhao', 'Mohan S. Kankanhalli']
['Zhanpeng Zhang', 'Ping Luo', 'Chen Change Loy', 'Xiaoou Tang']
['ZhouxiaWang', 'Tianshui Chen', 'Jimmy Ren', 'Weihao Yu', 'Hui Cheng', 'Liang Lin']
['Arushi Goel', 'Keng Teck Ma', 'Cheston T

In [216]:
cleaned_doi_set = set()
for paper in cleaned_arr:
    if(paper['DOI'][:2]=="10"):
        cleaned_doi_set.add(paper['DOI'])

print(len(cleaned_doi_set))
print(len(ref_doi_set))
for r in ref_doi_set:
    print(r)

221
3164
10.1016/j.knosys.2018.04.018
10.1007/978-0-387-09823-4_34
10.1007/s10955-006-9162-3
10.1137/1.9781611972832.44
10.1145/2063518.2063519
10.1145/988672.988727
10.1137/1.9781611974010.51
10.1016/j.ins.2010.11.022
10.1017/nws.2014.3
10.3115/v1/D14-1067
10.1016/j.knosys.2011.03.004
10.1145/1008992.1009039
10.1002/1098-2418(200103)18:2<116::AID-RSA1001>3.0.CO;2-2
10.1103/PhysRevLett.92.178701
10.7551/mitpress/9780262033589.001.0001
10.1145/1178677.1178709
10.24963/ijcai.2018/288
10.1145/2872427.2883041
10.1007/978-3-319-09265-2_37
10.1109/GreenCom-iThings-CPSCom.2013.174
10.1103/PhysRevE.77.046119
10.1038/nphys1746
10.1109/TPAMI.2013.220
10.1145/2124295.2124320
10.1145/1592665.1592675
10.1109/CVPRW.2009.5206492
10.1109/ICDM.2014.36
10.1093/bioinformatics/btl301
10.1007/978-3-540-69423-6_59
10.1109/ICDE.2008.4497502
10.7763/IJFCC.2013.V2.161
10.1016/j.jmva.2006.11.013
10.1073/pnas.0503018102
10.3115/1613715.1613857
10.1145/219717.219745
10.1093/acprof:oso/9780199206650.001.0001
10.10

10.1016/j.socnet.2008.08.001
10.1017/CBO9780511584176
10.1145/1553374.1553447
10.21236/ADA458734
10.1109/CVPR.2014.220
10.1109/ICDM.2008.89
10.1109/CVPR.2018.00895
10.1145/2953881
10.1145/2339530.2339738
10.1007/BF02854581
10.1137/1.9781611972740.43
10.1109/ICPR.2014.650
10.1137/1.9781611972887.11
10.1109/CVPR.2013.86
10.3115/v1/D14-1116
10.1007/s11768-016-5123-9
10.1109/12.508322
10.1016/j.bushor.2009.09.003
10.1109/CVPR.2018.00676
10.1109/ICDM.2014.69
10.1007/978-3-642-39159-0_1
10.1016/j.mri.2008.01.048
10.1007/s00265-003-0651-y
10.1145/1227161.1370597
10.1098/rsos.160863
10.1145/2187836.2187908
10.1007/978-3-540-24688-6_137
10.1109/NABIC.2010.5716360
10.14778/2735703.2735706
10.1016/j.socnet.2005.01.007
10.1080/15427951.2016.1177801
10.1038/35019019
10.1145/2320765.2320828
10.1088/1367-2630/16/6/063007
10.1145/2723372.2751523
10.1016/j.socnet.2008.03.005
10.1145/2750858.2806897
10.1109/TKDE.2018.2849727
10.1073/pnas.1116502109
10.1007/s00521-008-0225-z
10.1145/133160.133214
10.1145

10.1109/ASONAM.2012.184
10.1016/0925-2312(94)00012-H
10.1007/BF00332918
10.1145/2623330.2623645
10.1145/2492517.2492558
10.1007/978-3-642-15939-8_12
10.1109/TKDE.2007.250581
10.1109/TPAMI.2015.2505292
10.1145/2365934.2365945
10.1109/89.784104
10.18653/v1/D17-1037
10.14778/2367502.2367567
10.1209/0295-5075/105/68004
10.1109/TASL.2011.2134090
10.1007/978-3-540-33215-2_5
10.1145/1376916.1376928
10.1109/CVPR.2007.383497
10.1109/ICCV.2011.6126235
10.24963/ijcai.2019/264
10.1001/jama.1995.03520290054029
10.1145/1367497.1367583
10.1109/ICDM.2014.22
10.1109/ICCVW.2011.6130513
10.1109/TSMCB.2005.852980
10.1137/1.9781611972832.27
10.1088/1367-2630/10/5/053039
10.1093/imrn/rnx082
10.1145/2487575.2487691
10.1007/s10994-010-5214-7
10.1007/978-3-642-42042-9_53
10.1145/1401890.1402008
10.1177/1094670502004003003
10.1016/j.is.2014.05.003
10.1109/ICDE.2008.4497498
10.1016/j.telpol.2003.12.003
10.1371/journal.pone.0000165
10.1145/1645953.1646109
10.1103/RevModPhys.54.235
10.1002/9780470191613
10.1109/IC

In [92]:
for paper in cleaned_arr:
    if(paper['DOI'][:2]!='10'):
        print(paper['title'])

An End-to-End Network for Generating Social Relationship Graphs
Character Relationship Analysis in Movies Using Face Tracks
Audio-Visual Kinship Verification in the Wild
Social Relation Recognition from Videos via Multi-scale Spatial-Temporal Reasoning
Visual Social Relationship Recognition
Prediction of Personalized Rating by Combining Bandwagon Effect and Social Group Opinion: using Hadoop-Spark Framework
A hybrid collaborative ﬁltering model for social inﬂuence prediction in event-based social networks
Dynamic Network Embedding by Modeling Triadic Closure Process
EvolveGCN: Evolving Graph Convolutional Networks for Dynamic Graphs
Community Preserving Network Embedding
TIMERS: Error-Bounded SVD Restart on Dynamic Networks
Dynamic Network Embedding via Incremental Skip-gram with Negative Sampling
DepthLGP: Learning Embeddings of Out-of-Sample Nodes in Dynamic Networks
Toward an architecture for never-ending language learning
A three-way model for collective learning on multi-relationa

In [90]:
doi_cnt = 0
for c_doi in cleaned_doi_set:
    if(c_doi in ref_doi_set):
        doi_cnt += 1
print(doi_cnt)

101


In [ ]:
doi_cnt = 0
for c_doi in cleaned_arr:
    doi_cnt += 1
print(doi_cnt)

In [73]:
# for paper in yarr:
#     for ref in paper['references']:
#         if "cross" in ref.keys():
#             print(ref['cross'])
ref_doi_set = set()
print(yarr[0]['references'][3]['ref'])
for paper in yarr:
    for ref in paper['references']:
        try:
            ref_doi = json.loads(ref['cross'])['created']['DOI']
            ref_doi_set.add(ref_doi)
        except:
            print("error")
            
print(len(ref_doi_set))    
print(json.loads(yarr[0]['references'][4]['cross'])['created']['DOI'])

[4] Feld, Scott L. (1981). The Focused Organization of Social Ties. American Journal of Sociology. 86(5). 1015--1035.
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [78]:
print(len(ref_doi_set))
# for paper in yarr:
#     if(paper['DOI'][:2] == "10"):
#         print(paper['DOI'])
#         for paper['DOI'] in ref_doi_set:
#             print(">>>>>>>")

3164


In [79]:
print(ref_doi_set)

{'10.1016/j.knosys.2018.04.018', '10.1007/978-0-387-09823-4_34', '10.1007/s10955-006-9162-3', '10.1137/1.9781611972832.44', '10.1145/2063518.2063519', '10.1145/988672.988727', '10.1137/1.9781611974010.51', '10.1016/j.ins.2010.11.022', '10.1017/nws.2014.3', '10.3115/v1/D14-1067', '10.1016/j.knosys.2011.03.004', '10.1145/1008992.1009039', '10.1002/1098-2418(200103)18:2<116::AID-RSA1001>3.0.CO;2-2', '10.1103/PhysRevLett.92.178701', '10.7551/mitpress/9780262033589.001.0001', '10.1145/1178677.1178709', '10.24963/ijcai.2018/288', '10.1145/2872427.2883041', '10.1007/978-3-319-09265-2_37', '10.1109/GreenCom-iThings-CPSCom.2013.174', '10.1103/PhysRevE.77.046119', '10.1038/nphys1746', '10.1109/TPAMI.2013.220', '10.1145/2124295.2124320', '10.1145/1592665.1592675', '10.1109/CVPRW.2009.5206492', '10.1109/ICDM.2014.36', '10.1093/bioinformatics/btl301', '10.1007/978-3-540-69423-6_59', '10.1109/ICDE.2008.4497502', '10.7763/IJFCC.2013.V2.161', '10.1016/j.jmva.2006.11.013', '10.1073/pnas.0503018102', '1

In [ ]:
author_title = ""
for paper in all_arr:
    author_title = author_title +paper['title'] +"|"+str(paper['author']) +"\n"
save_tran_plain('author.txt',author_title)

In [ ]:
def data_clean(ori):
    for paper in ori:
        for key in full_keys:
            if key in paper.keys():
                if key == "author":
                    paper[key].replace('，',',').replace('\ufeff','')
                    
            else:
#                 缺失，填入默认值
                paper[key] = full_keys_default[key]
        if(paper['author'])

In [26]:
to_update = all_arr
for paper in to_update:
    for key in full_keys:
#         对全体可能的参数
        for ins in paper['inspect']:
            if(key in ins):
#                 对于每个错误
                if(key in paper.keys()):
#                 如果存在
                    print(paper['title'])
                    paper['journal']
                    print(paper['journal']+ str(search_ccf(paper['journal'])))

ccfClass
Measuring Social Networks with Digital Photograph Collections
ACM Conference on Hypertext and Hypermedia{'level': '', 'field': '', 'type': ''}
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
ccfClass
Family Member Identification from Photo Collections
WACV{'level': '', 'field': '', 'type': ''}
is_important
is_important
ccfClass
Learning deep compact similarity metric for kinship verification from face images
Information Fusion{'level': '', 'field': '', 'type': ''}
is_important
is_important
is_important
is_important
is_important
is_important
is_important
DOI
An End-to-End Network for Generating Social Relationship Graphs
CVPR{'level': 'A', 'field': '人工智能', 'type': '会议'}
is_important
is_important
is_important
is_important
is_important
is_important
ccfClass
Automatic Social Network Construction from Movies Using Film-Editing Cues
ICMEW{'level': 'B', 'field': '计算机图形学与多媒体', 'type': '会议'}
is_important
DOI
Character Relationship

ArXiv{'level': '', 'field': '', 'type': ''}
is_important
is_important
is_important
DOI
Community Preserving Network Embedding
The 31st AAAI Conference on Artificial Intelligence. 2017.{'level': 'A', 'field': '人工智能', 'type': '会议'}
is_important
ccfClass
DEEPEYE: Link Prediction in Dynamic Networks Based on Non-negative Matrix Factorization
Big data minging and analytics{'level': '', 'field': '', 'type': ''}
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
ccfClass
DHNE: Network Representation Learning Method for Dynamic Heterogeneous Networks
IEEE Access{'level': '', 'field': '', 'type': ''}
is_important
is_important
ccfClass
EpiRep: Learning Node Representations through Epidemic Dynamics on Networks
ACM International Conference on Web Intelligence{'level': 'C', 'field': '人工智能', 'type': '期刊'}
is_important
is_important
is_important
is_important
is_important
is_important
ccfClass
Deep Dynamic Netw

EMNLP{'level': 'B', 'field': '人工智能', 'type': '会议'}
citations(google scholar)
ccfClass
is_important
Knowledge graph and text jointly embedding
EMNLP{'level': 'B', 'field': '人工智能', 'type': '会议'}
DOI
Representation learning of knowledge graphs with entity descriptions
AAAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
month
Representation learning of knowledge graphs with entity descriptions
AAAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
citations(google scholar)
ccfClass
is_important
Representation learning of knowledge graphs with entity descriptions
AAAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
DOI
Text-enhanced representation learning for knowledge graph
IJCAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
month
Text-enhanced representation learning for knowledge graph
IJCAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
citations(google scholar)
ccfClass
is_important
Text-enhanced representation learning for knowledge graph
IJCAI{'level': 'A', 'field': '人工智能', 'type': '会议'}
month
J

SIGKDD{'level': 'A', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
references
Knowledge vault: A web-scale approach to probabilistic knowledge fusion
SIGKDD{'level': 'A', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
DOI
Probabilistic reasoning via deep learning: Neural association models
arXiv preprint{'level': '', 'field': '', 'type': ''}
month
Probabilistic reasoning via deep learning: Neural association models
arXiv preprint{'level': '', 'field': '', 'type': ''}
citations(google scholar)
ccfClass
is_important
Probabilistic reasoning via deep learning: Neural association models
arXiv preprint{'level': '', 'field': '', 'type': ''}
references
Probabilistic reasoning via deep learning: Neural association models
arXiv preprint{'level': '', 'field': '', 'type': ''}
month
Encoding temporal information for time-aware link prediction
EMNLP{'level': 'B', 'field': '人工智能', 'type': '会议'}
citations(google scholar)
ccfClass
is_important
Encoding temporal information for time-aware link prediction
EMNLP{'leve

Complexity{'level': '', 'field': '', 'type': ''}
is_important
references
A local random walk model for complex networks based on discriminative feature combinations
Expert Systems with Applications{'level': 'C', 'field': '人工智能', 'type': '期刊'}
is_important
is_important
is_important
is_important
references
Random Walk Decay Centrality
AAAI Conference on Artificial Intelligence. 2019{'level': 'A', 'field': '人工智能', 'type': '会议'}
is_important
is_important
is_important
DOI
Graph Representation Ensemble Learning
arXiv preprint arXiv:1909.02811{'level': '', 'field': '', 'type': ''}
ccfClass
Graph Representation Ensemble Learning
arXiv preprint arXiv:1909.02811{'level': '', 'field': '', 'type': ''}
is_important
ccfClass
Quantification of network structural dissimilarities
Nature communications{'level': '', 'field': '', 'type': ''}
is_important
DOI
Fast graph representation learning with PyTorch Geometric
International Conference on Learning Representations{'level': '', 'field': '', 'type': ''}


ACM International Conference on Information and Knowledge Management{'level': 'B', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
is_important
references
HeteroMed: Heterogeneous Information Network for Medical Diagnosis
ACM International Conference on Information and Knowledge Management{'level': 'B', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
is_important
references
Abnormal Event Detection via Heterogeneous Information Network Embedding
ACM International Conference on Information and Knowledge Management{'level': 'B', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfClass
is_important
ccfCl

WSDM{'level': 'B', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
DOI
Neural Graph Collaborative Filtering
SIGIR{'level': 'C', 'field': '人机交互与普适计算', 'type': '会议'}
ccfClass
Neural Graph Collaborative Filtering
SIGIR{'level': 'C', 'field': '人机交互与普适计算', 'type': '会议'}
is_important
references
Neural Graph Collaborative Filtering
SIGIR{'level': 'C', 'field': '人机交互与普适计算', 'type': '会议'}
DOI
Meta-Graph Based Recommendation Fusion over Heterogeneous Information Networks
Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining{'level': 'A', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
ccfClass
Meta-Graph Based Recommendation Fusion over Heterogeneous Information Networks
Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining{'level': 'A', 'field': '数据库／数据挖掘／内容检索', 'type': '会议'}
is_important
references
Meta-Graph Based Recommendation Fusion over Heterogeneous Information Networks
Proceedings of the 23rd ACM SIGKDD Internatio

In [41]:
# 数据处理
authors = set()
doi = set()
# for article in all_arr:
#     authors.update(article['author'])
for refs in all_arr[0]['references']:
    print(refs['ref'])
    print(get_crossref_info(refs['ref']))
    
#     doi.add(article['DOI'])
# authors_list = list(authors)
# authors_list.sort()
# print(authors_list)
print(doi)

[1] Adamic, Lada A. and Bernardo A. Huberman. (2002). Zipf's Law and the Internet. Glottometrics. 3. 143--150.
Resource not found.
Resource not found.
[2] Ames, Morgan. (2006). The Social Life of Snapshots. Thesis. UC Berkeley School of Information Mgt. and Systems.
Resource not found.
Resource not found.
[3] Marko Balabanović , Lonny L. Chu , Gregory J. Wolff, Storytelling with digital photographs, Proceedings of the SIGCHI conference on Human Factors in Computing Systems, p.564-571, April 01-06, 2000, The Hague, The Netherlands  [doi>10.1145/332040.332505]
{
 "issued": {
 "date-parts": [
 2000
]
},
 "link": [
 {
 "intended-application": "crawler-based",
 "content-version": "vor",
 "content-type": "unspecified",
 "URL": "http://dl.acm.org/ft_gateway.cfm?id=332505&amp;ftid=528&amp;dwn=1"
}
],
 "score": 1,
 "prefix": "http://id.crossref.org/10.1145",
 "author": [
 {
 "family": "Balabanovi\u0107",
 "given": "Marko"
},
 {
 "family": "Chu",
 "given": "Lonny L."
},
 {
 "family": "Wolff",
 "

{
 "license": [
 {
 "content-version": "tdm",
 "URL": "https://www.elsevier.com/tdm/userlicense/1.0/",
 "delay-in-days": 0,
 "date-parts": [
 2005,
 7,
 1
],
 "timestamp": 1120190400000
}
],
 "issued": {
 "date-parts": [
 2005,
 7
]
},
 "link": [
 {
 "intended-application": "text-mining",
 "content-version": "vor",
 "content-type": "text/xml",
 "URL": "https://api.elsevier.com/content/article/PII:S0378873305000092?httpAccept=text/xml"
},
 {
 "intended-application": "text-mining",
 "content-version": "vor",
 "content-type": "text/plain",
 "URL": "https://api.elsevier.com/content/article/PII:S0378873305000092?httpAccept=text/plain"
}
],
 "score": 1,
 "prefix": "http://id.crossref.org/10.1016",
 "author": [
 {
 "family": "Fu",
 "given": "Yang-chih"
}
],
 "container-title": "Social Networks",
 "citedby-count": 53,
 "reference-count": 30,
 "page": "169-186",
 "deposited": {
 "date-parts": [
 2019,
 1,
 28
],
 "timestamp": 1548674716000
},
 "created": {
 "date-parts": [
 2005,
 3,
 1
],
 "is

{
 "issued": {
 "date-parts": [
 2005,
 4
]
},
 "link": [
 {
 "intended-application": "crawler-based",
 "content-version": "vor",
 "content-type": "unspecified",
 "URL": "http://xplorestaging.ieee.org/ielx5/7756/30819/01427648.pdf?arnumber=1427648"
}
],
 "score": 1,
 "prefix": "http://id.crossref.org/10.1109",
 "author": [
 {
 "family": "Kindberg",
 "given": "T."
},
 {
 "family": "Spasojevic",
 "given": "M."
},
 {
 "family": "Fleck",
 "given": "R."
},
 {
 "family": "Sellen",
 "given": "A."
}
],
 "container-title": "IEEE Pervasive Computing",
 "citedby-count": 109,
 "reference-count": 12,
 "page": "42-50",
 "deposited": {
 "date-parts": [
 2017,
 3,
 15
],
 "timestamp": 1489548332000
},
 "created": {
 "date-parts": [
 2005,
 5,
 11
],
 "issue": "2",
 "title": "The Ubiquitous Camera: An In-Depth Study of Camera Phone Use",
 "subtitle": "",
 "type": "journal_article",
 "DOI": "10.1109/MPRV.2005.42",
 "ISSN": [
 "15361268"
],
 "URL": "https://doi.org/10.1109/MPRV.2005.42",
 "source": "Cros

KeyboardInterrupt: 

In [6]:
wos_arr = read_tran_json("wos.txt")
wos500_arr = read_tran_json("wos500.txt")

In [12]:
for paper in all_arr:
    print(paper['references'])

[{'ref': "[1] Adamic, Lada A. and Bernardo A. Huberman. (2002). Zipf's Law and the Internet. Glottometrics. 3. 143--150."}, {'ref': '[2] Ames, Morgan. (2006). The Social Life of Snapshots. Thesis. UC Berkeley School of Information Mgt. and Systems.'}, {'ref': '[3] Marko Balabanović , Lonny L. Chu , Gregory J. Wolff, Storytelling with digital photographs, Proceedings of the SIGCHI conference on Human Factors in Computing Systems, p.564-571, April 01-06, 2000, The Hague, The Netherlands  [doi>10.1145/332040.332505]'}, {'ref': '[4] Feld, Scott L. (1981). The Focused Organization of Social Ties. American Journal of Sociology. 86(5). 1015--1035.'}, {'ref': '[5] Ferligoj, Anuska and Valentina Hlebec. (1999). Evaluation of Social Network Measurement Instruments. Social Networks. 21. 111--130.'}, {'ref': '[6] Fu, Yang-chih. (2005). Measuring Personal Networks with Daily Contacts: A Single--Item Survey Question and the Contact Diary. Social Networks. 27. 169--186.'}, {'ref': '[7] Scott A. Golde

In [7]:
print(json.dumps(wos500_arr[0]))

{"PT": ["B"], "AU": ["Shang, Wenqian", "Shang, Songtao", "Feng, Shuchao", "Shi, Minyong"], "BE": ["Goto, T", "Xu, S", "Nguyen, N", "Li, W"], "TI": ["An Improved Video Recommendations Based on the Hyperlink-Graph Model"], "SO": ["2016 4TH INTL CONF ON APPLIED COMPUTING AND INFORMATION TECHNOLOGY/3RD", "INTL CONF ON COMPUTATIONAL SCIENCE/INTELLIGENCE AND APPLIED", "INFORMATICS/1ST INTL CONF ON BIG DATA, CLOUD COMPUTING, DATA SCIENCE &", "ENGINEERING (ACIT-CSII-BCD)"], "BP": ["379"], "EP": ["383"], "DI": ["10.1109/ACIT-CSII-BCD.2016.80"], "PD": ["2016"], "PY": ["2016"], "AB": ["The graph-based algorithm for personalized recommendations mainly", "depends on the user-item model to construct a bipartite graph. We can", "provide recommendations by analyzing the bipartite graphs. However, for", "personalized videos recommendations, the classical recommendation", "algorithm based on graphs has low efficiency. Therefore, this paper", "gives an improved video recommendation algorithm that is base

In [5]:
# 数据写入
with open( TRANSFORMED_DIR + OUTPUT,'w') as o:
    o.write(json.dumps(all_arr,ensure_ascii=False ))

In [12]:
# 数据处理并写入 group by topic
gbts_arr = []
for fname in addr_list:
    gbt_obj = {}
    addr = RAW_DIR + fname
#     添加话题
    en_topic,cn_topic = fname.replace(".txt","").split("_")
    topic = {"en":en_topic,"cn":cn_topic}
    gbt_obj["topic"] = topic
#     话题下论文
    with open(addr,'r') as f:
        farr = json.load(f)
        gbt_obj["papers"] = farr
        

    gbts_arr.append(dict(gbt_obj))
    
with open(TRANSFORMED_DIR + GBTOUTPUT,'w') as o:
    o.write(json.dumps(gbts_arr[2],ensure_ascii=False))


In [17]:
# 数据处理并写入
title_arr = []
for fname in addr_list:
    addr = RAW_DIR + fname
    with open(addr,'r') as f:
        farr = json.load(f)
        for fobj in farr:
            title_arr.append(str(fobj["title"]))

save_tran_json(TITLEOUTPUT,title_arr)
print(len(title_arr))


361


In [17]:
for paper in wos_arr:
    print(paper.keys())
    try:
        print(paper['DI'])
    except:
        pass

dict_keys(['PT', 'AU', 'AF', 'GP', 'TI', 'SO', 'LA', 'DT', 'CT', 'CY', 'CL', 'SP', 'DE', 'AB', 'C1', 'RP', 'EM', 'CR', 'NR', 'TC', 'Z9', 'U1', 'U2', 'PU', 'PI', 'PA', 'BN', 'PY', 'BP', 'EP', 'DI', 'PG', 'WC', 'SC', 'GA', 'UT', 'DA'])
['10.1109/GreenCom.2012.72']
dict_keys(['PT', 'AU', 'AF', 'TI', 'SO', 'LA', 'DT', 'DE', 'AB', 'C1', 'RP', 'EM', 'FU', 'FX', 'CR', 'NR', 'TC', 'Z9', 'U1', 'U2', 'PU', 'PI', 'PA', 'SN', 'EI', 'J9', 'JI', 'PY', 'VL', 'IS', 'BP', 'EP', 'DI', 'PG', 'WC', 'SC', 'GA', 'UT', 'DA'])
['10.3233/IDA-2011-0486']
dict_keys(['PT', 'AU', 'AF', 'TI', 'SO', 'LA', 'DT', 'DE', 'ID', 'AB', 'C1', 'RP', 'EM', 'OI', 'FU', 'FX', 'CR', 'NR', 'TC', 'Z9', 'U1', 'U2', 'PU', 'PI', 'PA', 'SN', 'J9', 'JI', 'PD', 'PY', 'VL', 'IS', 'SI', 'BP', 'EP', 'DI', 'PN', 'PG', 'WC', 'SC', 'GA', 'UT', 'DA'])
['10.1109/TMM.2012.2187436']
dict_keys(['PT', 'AU', 'AF', 'GP', 'TI', 'SO', 'SE', 'LA', 'DT', 'CT', 'CY', 'CL', 'SP', 'AB', 'C1', 'RP', 'EM', 'RI', 'FU', 'FX', 'CR', 'NR', 'TC', 'Z9', 'U1', 'U2',

In [221]:
len(list_tran_arr)

453

In [ ]:
doi_
for 